# SVM

This files includes usage of svm for discrete condition.

---

## Intro

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection. In this file, we only introduce the svm for classifcation.

---

## Math Principle

Here is the recommended [video](https://www.youtube.com/watch?v=_PwhiWxHK8o)


---

## Advantage & Disadvantage

- Advantage
  - Effective in high dimensional spaces.
  - Still effective in cases where number of dimensions is greater than the number of samples.
  - Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
  - Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.
- Disadvantage
  - If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
  - SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).